In [ ]:
categorical = [
    'age': 8,
    'sex': 3,
    'user_lv_cd': 6,
    'user_id': 100000,
    'sku_id': 30000, # only in P , others set 0
    'brand':10
    'cate':442
    'model_id': 100
    
    
    'time':
    ''
    
    'dt': 10
    'a1':10
    'a2':10
    'a3':10
    'comment_num':10
    'has_bad_comment': 2
    
]


In [1]:
from collections import defaultdict
import json

In [2]:
P = set()
with open('../data/csv/JData_Product.csv') as f:
    for i, line in enumerate(f):
        if i == 0: continue
        items = line.strip().split(',')
        P.add(items[0])

In [3]:
res = defaultdict(list)
with open('../data/csv/JData_Action_All.csv') as f:
    for i, line in enumerate(f):
        items = line.strip().split(',')
        if items[1] in P:
            res[(items[0], items[1])].append(items)

In [4]:
import datetime
d = None

seconds_in_day = 3600 * 24
seconds_marks = [3600, 3 * 3600, 15 * 3600, 31 * 3600, 63 * 3600, 127 * 3600, 255 * 2600, 511 * 3600, 1023 * 3600, float('inf')]
start_date = datetime.datetime.strptime('2016-01-01', '%Y-%m-%d')
def time2ds(time):
    delta = (datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') - start_date)
    return delta.days, delta.seconds
def time2s(time):
    d, s = time2ds(time)
    return d * seconds_in_day + s

In [5]:
from collections import defaultdict
def transform(key, a, start_ts, end_ts):
    a.sort(key=lambda o: o[2])
    
    info = {'user_id': key[0], 'sku_id': key[1], 'label': 0, 'interval_actions': [defaultdict(int) for i in range(len(seconds_marks))]} 
    k = len(seconds_marks) - 1
    for i, aa in enumerate(a):
        ts = time2s(aa[2])
        if ts < start_ts:
            while k >= 1 and start_ts - ts <= seconds_marks[k - 1]: k -= 1
            name = aa[4]
            #if aa[4] == '6': name = '6_%s' % aa[3]
            info['interval_actions'][k][name] += 1 
        elif ts < end_ts:
            if aa[4] == '4': info['label'] = 1
        else:
            pass
    return info

In [6]:
from __future__ import print_function
with open('../data/action_11_16.json', 'w') as f:
    for i, (k, v) in enumerate(res.iteritems()):
        obj = transform(k, v, time2s('2016-04-11 00:00:00'), time2s('2016-04-16 00:00:00'))
        print(json.dumps(obj), file=f)

In [7]:
with open('../data/action_16_21.json', 'w') as f:
    for i, (k, v) in enumerate(res.iteritems()):
        obj = transform(k, v, time2s('2016-04-16 00:00:00'), time2s('2016-04-21 00:00:00'))
        print(json.dumps(obj), file=f)

In [8]:
with open('../data/action_06_11.json', 'w') as f:
    for i, (k, v) in enumerate(res.iteritems()):
        obj = transform(k, v, time2s('2016-04-06 00:00:00'), time2s('2016-04-11 00:00:00'))
        print(json.dumps(obj), file=f)

In [9]:
with open('../data/action_04_09.json', 'w') as f:
    for i, (k, v) in enumerate(res.iteritems()):
        obj = transform(k, v, time2s('2016-04-04 00:00:00'), time2s('2016-04-09 00:00:00'))
        print(json.dumps(obj), file=f)